In [1]:
pip install selenium

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [2]:
pip install webdriver_manager

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [3]:
pip install WebDriverWait

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
ERROR: Could not find a version that satisfies the requirement WebDriverWait (from versions: none)
ERROR: No matching distribution found for WebDriverWait


In [4]:
from selenium import webdriver

In [5]:
# selenium으로 무엇인가 입력하기 위한 import
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
# 페이지 로딩을 기다리는데에 사용할 time 모듈 import
import time

In [6]:
# 크롬드라이버 실행
driver = webdriver.Chrome() 

In [7]:
driver.quit()

In [6]:
#chrome driver 설치 코드
def get_chrome_driver():
    #1. 크롬 옵션 세팅
    chrome_options = webdriver.ChromeOptions()
    #2.driver 생성
    driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=chrome_options
)

    return driver

- webdriver : 파이썬과 브라우저를 연결하기 위한 객체
- Service : [최신버전] 어떤 브라우저를 사용하게 할건지 선택
- ChromeDriverManager : [최신버전] Chrome을 사용하기 위한 드라이버 설정

In [9]:
driver = get_chrome_driver()

## 1. 타자 연봉 크롤링

In [12]:
driver = get_chrome_driver()

driver.get("https://www.koreabaseball.com/Record/Player/HitterBasic/Basic1.aspx")
driver.implicitly_wait(1)
# "HT","SS","LG","OB",
hitter_name_list = []
hitter_salary_list = []
team = ["HT","SS","LG","OB","SK","KT","HH","LT","NC","WO"]


for t in team: 
    
    for i in range(1,26):
        # 드롭다운
        dropdown = "ctl00$ctl00$ctl00$cphContents$cphContents$cphContents$ddlTeam$ddlTeam"
        select_element = driver.find_element(By.NAME, dropdown)
        select = Select(select_element)
        select.select_by_value(t)  

        time.sleep(2) # 페이지의 완벽한 로딩을 위한 작업
    
        # 이름 버튼 눌러서 상세 페이지 열기
        name_css_selector = f'#cphContents_cphContents_cphContents_udpContent > div.record_result > table > tbody > tr:nth-child({i}) > td:nth-child(2) > a'
        button = driver.find_element(By.CSS_SELECTOR,name_css_selector)
        ActionChains(driver).click(button).perform()

            
        name = driver.find_elements(By.ID, "cphContents_cphContents_cphContents_playerProfile_lblName")
        salary = driver.find_elements(By.ID, "cphContents_cphContents_cphContents_playerProfile_lblSalary")
            
        hitter_name_list.append(name[0].get_attribute('innerHTML'))
        hitter_salary_list.append(salary[0].get_attribute('innerHTML'))
            
            
        driver.back()


In [ ]:
hitter_name_list


In [ ]:
hitter_salary_list


In [ ]:
import pandas as pd

# 데이터 프레임으로 만들기
df_hitter = pd.DataFrame({"선수명":hitter_name_list, "salary":hitter_salary_list})

In [ ]:
df_hitter.shape

In [ ]:
# 데이터 저장
df_hitter.to_csv('./hitter_salary.csv',index=False)

## 2. 투수 연봉 크롤링

In [ ]:
driver = get_chrome_driver()

driver.set_window_size(2000, 1000)
driver.set_window_position(1,0) 
driver.get("https://www.koreabaseball.com/Record/Player/PitcherBasic/BasicOld.aspx")
driver.implicitly_wait(1)
# "HT","SS","LG","OB",
name_list = []
salary_list = []
team = ["HT","SS","LG","OB","SK","KT","HH","LT","NC","WO"]

time.sleep(3) 

for t in team: 
    
    for i in range(1,26):
            
        # 드롭다운
        dropdown = "ctl00$ctl00$ctl00$cphContents$cphContents$cphContents$ddlTeam$ddlTeam"
        select_element = driver.find_element(By.NAME, dropdown)
        select = Select(select_element)
        select.select_by_value(t)
    
        time.sleep(2) 

        # 이름 버튼 눌러서 상세 페이지 열기
        name_css_selector = f'#cphContents_cphContents_cphContents_udpContent > div.record_result > table > tbody > tr:nth-child({i}) > td:nth-child(2) > a'
        button = driver.find_element(By.CSS_SELECTOR,name_css_selector)

        ActionChains(driver).click(button).perform()
    
        name_x = '//*[@id="cphContents_cphContents_cphContents_playerProfile_lblName"]'
        salary_x = '//*[@id="cphContents_cphContents_cphContents_playerProfile_lblSalary"]'
            
        name = driver.find_elements(By.XPATH,name_x)
        salary = driver.find_elements(By.XPATH, salary_x)

        pitcher_name_list.append(name[0].get_attribute('innerHTML'))
        pitcher_salary_list.append(salary[0].get_attribute('innerHTML'))
            
        driver.back()


In [ ]:
pitcher_salary_list

In [ ]:
pitcher_name_list

In [ ]:
df_salary = pd.DataFrame({"선수명":pitcher_name_list, "salary":pitcher_salary_list})

In [ ]:
# 데이터 저장
df_salary.to_csv('./pitcher_salary.csv',index=False)

# 3. 타자의 키와 몸무게

In [16]:
driver = get_chrome_driver()

driver.set_window_size(2000, 1000)
driver.set_window_position(1,0) 
driver.get("https://www.koreabaseball.com/Record/Player/HitterBasic/Basic1.aspx")
driver.implicitly_wait(1)
# "HT","SS","LG","OB",
hitter_name_list = []
hitter_H_W_list = []
hitter_team_list = []
team = ["HT","SS","LG","OB","SK","KT","HH","LT","NC","WO"]

time.sleep(3) 

for t in team: 
    
    for i in range(1,26):
            
        # 드롭다운
        dropdown = "ctl00$ctl00$ctl00$cphContents$cphContents$cphContents$ddlTeam$ddlTeam"
        select_element = driver.find_element(By.NAME, dropdown)
        select = Select(select_element)
        select.select_by_value(t)
    
        time.sleep(2) 

        # 이름 버튼 눌러서 상세 페이지 열기
        name_css_selector = f'#cphContents_cphContents_cphContents_udpContent > div.record_result > table > tbody > tr:nth-child({i}) > td:nth-child(2) > a'
        button = driver.find_element(By.CSS_SELECTOR,name_css_selector)

        ActionChains(driver).click(button).perform()
    
        name_x = '//*[@id="cphContents_cphContents_cphContents_playerProfile_lblName"]'
        h_w_x = '//*[@id="cphContents_cphContents_cphContents_playerProfile_lblHeightWeight"]'
        team_x = '//*[@id="contents"]/div[2]/div[2]/div[2]/table/tbody/tr/td[1]'
            
        name = driver.find_elements(By.XPATH,name_x)
        H_W = driver.find_elements(By.XPATH, h_w_x)
        team = driver.find_elements(By.XPATH, team_x)

        hitter_name_list.append(name[0].get_attribute('innerHTML'))
        hitter_H_W_list.append(H_W[0].get_attribute('innerHTML'))
        hitter_team_list.append(team[0].get_attribute('innerHTML'))
            
        driver.back()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#cphContents_cphContents_cphContents_udpContent > div.record_result > table > tbody > tr:nth-child(24) > td:nth-child(2) > a"}
  (Session info: chrome=127.0.6533.122); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00308923+23283]
	(No symbol) [0x002CE934]
	(No symbol) [0x00200733]
	(No symbol) [0x0024326F]
	(No symbol) [0x002434AB]
	(No symbol) [0x0027EE42]
	(No symbol) [0x00264464]
	(No symbol) [0x0027CB8D]
	(No symbol) [0x002641B6]
	(No symbol) [0x00238017]
	(No symbol) [0x0023890D]
	GetHandleVerifier [0x003FA5F3+1013699]
	GetHandleVerifier [0x00403E4C+1052700]
	GetHandleVerifier [0x003FD4B4+1025668]
	GetHandleVerifier [0x0032EA2B+179195]
	(No symbol) [0x002D6833]
	(No symbol) [0x002D3198]
	(No symbol) [0x002D3337]
	(No symbol) [0x002CB4BE]
	

In [17]:
hitter_H_W_list

['183cm/73kg',
 '174cm/83kg',
 '183cm/85kg',
 '184cm/95kg',
 '182cm/95kg',
 '188cm/93kg',
 '185cm/95kg',
 '178cm/72kg',
 '178cm/72kg',
 '165cm/77kg',
 '184cm/83kg',
 '183cm/100kg',
 '178cm/85kg',
 '178cm/100kg',
 '180cm/106kg',
 '175cm/68kg',
 '176cm/84kg',
 '184cm/84kg',
 '182cm/92kg',
 '173cm/85kg',
 '172cm/82kg',
 '178cm/85kg',
 '185cm/90kg',
 '177cm/76kg',
 '182cm/90kg',
 '177cm/80kg',
 '188cm/105kg',
 '185cm/83kg',
 '189cm/75kg',
 '163cm/64kg',
 '182cm/93kg',
 '185cm/100kg',
 '188cm/101kg',
 '174cm/81kg',
 '180cm/82kg',
 '183cm/81kg',
 '181cm/75kg',
 '178cm/82kg',
 '180cm/87kg',
 '176cm/68kg',
 '180cm/81kg',
 '178cm/78kg',
 '185cm/107kg',
 '163cm/62kg',
 '177cm/78kg',
 '185cm/85kg',
 '182cm/93kg',
 '182cm/85kg',
 '184cm/89kg',
 '184cm/84kg',
 '181cm/85kg',
 '174cm/76kg',
 '172cm/70kg',
 '177cm/80kg',
 '175cm/78kg',
 '189cm/94kg',
 '183cm/97kg',
 '182cm/88kg',
 '188cm/105kg',
 '171cm/67kg',
 '178cm/110kg',
 '178cm/92kg',
 '177cm/75kg',
 '180cm/75kg',
 '185cm/80kg',
 '186cm/85kg',
 

In [18]:
hitter_name_list

['김규성',
 '한승택',
 '김도영',
 '한준수',
 '이우성',
 '소크라테스',
 '변우혁',
 '홍종표',
 '박찬호',
 '김선빈',
 '고종욱',
 '나성범',
 '최원준',
 '황대인',
 '최형우',
 '박정우',
 '서건창',
 '박민',
 '김태군',
 '이창진',
 '예진원',
 '김호령',
 '곽도규',
 '김두현',
 '김사윤',
 '김민수',
 '디아즈',
 '카데나스',
 '구자욱',
 '김지찬',
 '윤정빈',
 '강민호',
 '맥키넌',
 '김헌곤',
 '이재현',
 '김영웅',
 '류지혁',
 '이성규',
 '이병헌',
 '안주형',
 '김재상',
 '김현준',
 '박병호',
 '김성윤',
 '김태훈',
 '김재성',
 '전병우',
 '김재혁',
 '이창용',
 '김동진',
 '이영빈',
 '최원영',
 '김대원',
 '최명경',
 '문성주',
 '홍창기',
 '오스틴',
 '문보경',
 '김현수',
 '신민재',
 '김범석',
 '박동원',
 '구본혁',
 '박해민',
 '오지환',
 '김현종',
 '김주성',
 '안익훈',
 '함창건',
 '허도환',
 '김성진',
 '최승민',
 '김민수',
 '송찬의',
 '김대현',
 '전다민',
 '제러드',
 '홍성호',
 '허경민',
 '양의지',
 '라모스',
 '이유찬',
 '김기연',
 '김재호',
 '강승호',
 '정수빈',
 '조수행',
 '김재환',
 '전민재',
 '양석환',
 '서예일',
 '윤준호',
 '장규빈',
 '박준영',
 '김민혁',
 '박계범',
 '안승한',
 '장승현',
 '김인태',
 '양찬열',
 '에레디아',
 '박지환',
 '정준재',
 '추신수',
 '최정',
 '박성한',
 '이지영',
 '최지훈',
 '고명준',
 '최준우',
 '오태곤',
 '하재훈',
 '조형우',
 '한유섬',
 '김성현',
 '김민식',
 '정현승',
 '강진성',
 '안상현',
 '최경모',
 '최상민',
 '전의산',
 '김찬형',
 '김창평',
 '류효승'

In [19]:
hitter_team_list

['KIA',
 'KIA',
 'KIA',
 'KIA',
 'KIA',
 'KIA',
 'KIA',
 'KIA',
 'KIA',
 'KIA',
 'KIA',
 'KIA',
 'KIA',
 'KIA',
 'KIA',
 'KIA',
 'KIA',
 'KIA',
 'KIA',
 'KIA',
 'KIA',
 'KIA',
 'KIA',
 'KIA',
 'KIA',
 '삼성',
 '삼성',
 '삼성',
 '삼성',
 '삼성',
 '삼성',
 '삼성',
 '삼성',
 '삼성',
 '삼성',
 '삼성',
 '삼성',
 '삼성',
 '삼성',
 '삼성',
 '삼성',
 '삼성',
 '삼성',
 '삼성',
 '삼성',
 '삼성',
 '삼성',
 '삼성',
 '삼성',
 '삼성',
 'LG',
 'LG',
 'LG',
 'LG',
 'LG',
 'LG',
 'LG',
 'LG',
 'LG',
 'LG',
 'LG',
 'LG',
 'LG',
 'LG',
 'LG',
 'LG',
 'LG',
 'LG',
 'LG',
 'LG',
 'LG',
 'LG',
 'LG',
 'LG',
 'LG',
 '두산',
 '두산',
 '두산',
 '두산',
 '두산',
 '두산',
 '두산',
 '두산',
 '두산',
 '두산',
 '두산',
 '두산',
 '두산',
 '두산',
 '두산',
 '두산',
 '두산',
 '두산',
 '두산',
 '두산',
 '두산',
 '두산',
 '두산',
 '두산',
 '두산',
 'SSG',
 'SSG',
 'SSG',
 'SSG',
 'SSG',
 'SSG',
 'SSG',
 'SSG',
 'SSG',
 'SSG',
 'SSG',
 'SSG',
 'SSG',
 'SSG',
 'SSG',
 'SSG',
 'SSG',
 'SSG',
 'SSG',
 'SSG',
 'SSG',
 'SSG',
 'SSG',
 'SSG',
 'SSG',
 'KT',
 'KT',
 'KT',
 'KT',
 'KT',
 'KT',
 'KT',
 'KT',
 'KT',
 'KT',
 'KT'

In [21]:
import pandas as pd

In [22]:
df_hw = pd.DataFrame({"선수명":hitter_name_list, "H_W":hitter_H_W_list, "팀명":hitter_team_list})

In [25]:
df_hw.to_csv('./Hight_Weight.csv',index=False)

# 4. 타자의 나이와 입단년도

In [23]:
driver = get_chrome_driver()

driver.set_window_size(2000, 1000)
driver.set_window_position(1,0) 
driver.get("https://www.koreabaseball.com/Record/Player/HitterBasic/Basic1.aspx")
driver.implicitly_wait(1)

hitter_name_list = []
hitter_birth_list = []
hitter_debut_list = []
hitter_team_list = []

# team = ["HT","SS","LG","OB","SK","KT","HH","LT","NC","WO"]
team = ["NC","WO"]

time.sleep(3) 

for t in team: 
    
    for i in range(1,26):
            
        # 드롭다운
        dropdown = "ctl00$ctl00$ctl00$cphContents$cphContents$cphContents$ddlTeam$ddlTeam"
        select_element = driver.find_element(By.NAME, dropdown)
        select = Select(select_element)
        select.select_by_value(t)
    
        time.sleep(2) 

        # 이름 버튼 눌러서 상세 페이지 열기
        name_css_selector = f'#cphContents_cphContents_cphContents_udpContent > div.record_result > table > tbody > tr:nth-child({i}) > td:nth-child(2) > a'
        button = driver.find_element(By.CSS_SELECTOR,name_css_selector)

        ActionChains(driver).click(button).perform()

        # copy_XPATH
        name_x = '//*[@id="cphContents_cphContents_cphContents_playerProfile_lblName"]'
        debut_x = '//*[@id="cphContents_cphContents_cphContents_playerProfile_lblJoinInfo"]'
        birth_x = '//*[@id="cphContents_cphContents_cphContents_playerProfile_lblBirthday"]'
        team_x = '//*[@id="contents"]/div[2]/div[2]/div[2]/table/tbody/tr/td[1]'
            
        name = driver.find_elements(By.XPATH,name_x)
        birth = driver.find_elements(By.XPATH, birth_x)
        debut = driver.find_elements(By.XPATH, debut_x)
        team = driver.find_elements(By.XPATH, team_x)

        hitter_name_list.append(name[0].get_attribute('innerHTML'))
        hitter_birth_list.append(birth[0].get_attribute('innerHTML'))
        hitter_debut_list.append(debut[0].get_attribute('innerHTML'))
        hitter_team_list.append(team[0].get_attribute('innerHTML'))
            
        driver.back()


In [24]:
hitter_name_list

['박영빈',
 '박건우',
 '박민우',
 '김한별',
 '손아섭',
 '천재환',
 '서호철',
 '권희동',
 '데이비슨',
 '박세혁',
 '김휘집',
 '최정원',
 '김주원',
 '김수윤',
 '한석현',
 '박시원',
 '김성욱',
 '송승환',
 '오영수',
 '김형준',
 '도태훈',
 '박한결',
 '김세훈',
 '김영규',
 '김재열',
 '송성문',
 '김혜성',
 '도슨',
 '이용규',
 '변상권',
 '이주형',
 '김건희',
 '고영우',
 '원성준',
 '김재현',
 '임병욱',
 '최주환',
 '이원석',
 '이형종',
 '김태진',
 '장재영',
 '박수종',
 '김동헌',
 '김병휘',
 '주성원',
 '이재상',
 '김시앙',
 '김주형',
 '박성빈',
 '김웅빈']

In [25]:
import pandas as pd
df_bday_debut2 = pd.DataFrame({"선수명":hitter_name_list, "birth":hitter_birth_list, "팀명":hitter_team_list, "입단년도":hitter_debut_list})


In [22]:
df_bday_debut

,선수명,birth,팀명,입단년도
0,김규성,1997년 03월 08일,KIA,17KIA
1,한승택,1994년 06월 21일,KIA,13한화
2,김도영,2003년 10월 02일,KIA,22KIA
3,한준수,1999년 02월 13일,KIA,18KIA
4,이우성,1994년 07월 17일,KIA,13두산
...,...,...,...,...
216,김성욱,1993년 05월 01일,NC,12NC
217,송승환,2000년 10월 28일,NC,19두산
218,오영수,2000년 01월 30일,NC,18NC
219,김형준,1999년 11월 02일,NC,18NC


In [26]:
df_bday_debut_all = pd.concat([df_bday_debut,df_bday_debut2])

In [27]:
df_bday_debut_all

,선수명,birth,팀명,입단년도
0,김규성,1997년 03월 08일,KIA,17KIA
1,한승택,1994년 06월 21일,KIA,13한화
2,김도영,2003년 10월 02일,KIA,22KIA
3,한준수,1999년 02월 13일,KIA,18KIA
4,이우성,1994년 07월 17일,KIA,13두산
...,...,...,...,...
45,이재상,2005년 04월 17일,키움,24키움
46,김시앙,2001년 10월 31일,키움,22키움
47,김주형,1996년 03월 05일,키움,19키움
48,박성빈,2004년 04월 21일,키움,24키움


In [38]:
df_bday_debut_all['입단년도'] = df_bday_debut_all['입단년도'].str[:2].astype('int64')

In [39]:
df_bday_debut_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 271 entries, 0 to 49
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   선수명     271 non-null    object
 1   birth   271 non-null    object
 2   팀명      271 non-null    object
 3   입단년도    271 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 10.6+ KB


In [42]:
df_bday_debut_all['연차'] = 24 - df_bday_debut_all['입단년도'] + 1

In [43]:
df_bday_debut_all['연차']

0      8
1     12
2      3
3      7
4     12
      ..
45     1
46     3
47     6
48     1
49     9
Name: 연차, Length: 271, dtype: int64

In [46]:
df_bday_debut_all.to_csv('bday_debut.csv',index=False)